In [58]:
import pandas as pd
import numpy as np
import datetime
data = pd.read_csv("../../data/wetterdaten/Rohdaten/data-test.csv")
for key in data.keys():
    pd.to_numeric(key, errors="coerce")
data["time"] = pd.to_datetime(data["time"], unit="s")
to_process = pd.read_csv("../../data/listen_aus_bad/daten-andere-räume.csv")
#to_process = pd.read_csv("../../data/listen_aus_bad/test-bad-mitte.csv")
vent_old = pd.read_csv("../../data/wetterdaten/verarbeitet/vent.csv",sep=";")

In [25]:
data

,Unnamed: 0,temperature,humidity,time,hum_out,temp_out,humidity_oben,temperature_oben
0,0,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,1,NaN,NaN,2023-03-09 03:13:37.295016960,NaN,10,NaN,NaN
2,2,NaN,NaN,2023-03-09 03:14:07.014694912,90,10,NaN,NaN
3,3,NaN,NaN,2023-03-09 03:18:37.341829120,90,10,NaN,NaN
4,4,NaN,NaN,2023-03-09 03:19:07.025666048,89,10,NaN,NaN
...,...,...,...,...,...,...,...,...
5977,5977,22.1,51,2023-03-19 12:36:21.409672960,50,17,unavailable,unavailable
5978,5978,22.1,51,2023-03-19 12:36:21.410091008,50,17,unavailable,21.0
5979,5979,22.1,51,2023-03-19 12:36:21.534496000,50,17,57,21.0
5980,5980,22.1,51,2023-03-19 12:37:32.520086016,50,unavailable,57,21.0


In [59]:
# for venting
vent_now = []

current_ID = 0

for i in range(len(to_process)):
    year, month = 2023, 3
    day_start, day_end = to_process["Tag"][i], to_process["Tag"][i]
    
    start = datetime.datetime(year, month, day_start, to_process["Stunde"][i], to_process["Beginn"][i])
    end = datetime.datetime(year, month, day_end, to_process["Stunde Ende"][i], to_process["Ende"][i])
    time_mask = (data['time'] >= start) & \
            (data['time'] <= end)
    vent_now.append(data[time_mask].copy())
    vent_now[-1]["Id"] = current_ID
    current_ID += 1

vent_now = pd.concat(vent_now, ignore_index=True)

In [60]:
id = -1
local_time = []
for i in range(len(vent_now)):  
    if vent_now.iloc[i]["Id"] != id:
        id = vent_now.iloc[i]["Id"]
        first_time = vent_now.iloc[i]["time"]
        time = 0
    else:
        time = (vent_now.iloc[i]["time"] - first_time).seconds//60
        
    local_time.append(time)
vent_now["local time"] = local_time

In [61]:
def e_s(T): # Source https://journals.ametsoc.org/view/journals/bams/86/2/bams-86-2-225.xml?tab_body=pdf Equation 6 p.226
    C_1 = 610.94 #Kp
    A_1 =  17.625
    B_1 =243.04 #°C
    return C_1*np.exp((A_1*T)/(B_1+T))

def h(h_0, h_out, F_größe, R_größe, t, k_1, k_2):  # The absolute humidity model
    return (h_0 -h_out)/((t+1)**((k_1*F_größe)/(k_2*R_größe))) + h_out

def T(T_0, T_out, F_größe, R_größe, t, k_1, k_2): # The temperature model
    return (T_0-T_out)/((t+1)**((k_1*F_größe)/(k_2*R_größe))) + T_out

def model_h(window, room, L_out, L_begin, T_out, T_begin,t, k1,k2,k3,k4, training=False):
    L_abs_begin = (L_begin/100)*e_s(T_begin)
    L_abs_out = (L_out/100)*e_s(T_out)
    pred_T = T(T_begin, T_out, window, room, t, k3, k4)
    pred_L = h(L_abs_begin, L_abs_out, window, room, t, k1, k2)
    result= (pred_L/e_s(pred_T))*100
    
    if training:
        to_return = []
        for res in result:
            if res >100:
                to_return.append(-np.inf)
            else:
                to_return.append(res)
        return to_return
    return result
constants = [49.9737675 ,  2.28452262, 49.3679433 ,  8.39747826]

In [62]:
sum_of_all = 0
count = 0
maximum = 0
vent_now = vent_now.dropna()
erg = []
for i in vent_now.iterrows():
    try:  # unavailable could break the int we skip those lines
        right_serie = vent_now[vent_now["local time"] == 0][vent_now["Id"] == i[1]["Id"]].iloc[0]
        current = int(i[1]["humidity_oben"])-model_h(1.52, 30, int(right_serie["hum_out"]), int(right_serie["humidity_oben"]), float(right_serie["temp_out"]), float(right_serie["temperature_oben"]), int(i[1]["local time"]), *constants)
        maximum = max(maximum, abs(current))
        erg.append(pd.DataFrame({"error": [abs(current)]}))
        sum_of_all += abs(current)
        count +=1
    except: continue
print("Maximum:", maximum)
print("Average:", sum_of_all/count)
pd.concat(erg,ignore_index=True).describe()

Maximum: 20.381835258931822
Average: 4.899839181592993


/tmp/ipykernel_6276/1839097122.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  right_serie = vent_now[vent_now["local time"] == 0][vent_now["Id"] == i[1]["Id"]].iloc[0]


,error
count,29.000000
mean,4.899839
std,6.539654
min,0.000000
25%,0.000000
50%,1.626077
75%,6.249761
max,20.381835


In [36]:
# With the trainingsdata
vent_now = vent_old
vent_now["humidity"] = vent_now["humidity_bathroom_middle"]
vent_now["temperature"] = vent_now["temperature_bathroom_middle"]
vent_now["hum_out"] = vent_now["humidity_outside"]
vent_now["temp_out"] = vent_now["temperature_outside"]
vent_now[vent_now["room"]=="mid"]
vent_now = vent_now.dropna()

erg = []

sum_of_all = 0
count = 0
maximum = 0
for i in vent_now.iterrows():
    right_serie = vent_now[vent_now["local time"] == 0][vent_now["Id"] == i[1]["Id"]]
    if len(right_serie)==0:
        continue
    right_serie=right_serie.iloc[0]
    current = int(i[1]["humidity"])-model_h(1.52, 30, int(right_serie["hum_out"]), int(right_serie["humidity"]), float(right_serie["temp_out"]), float(right_serie["temperature"]), int(i[1]["local time"]), *constants)
    if not np.isnan(current):
        maximum = max(maximum, abs(current))
        sum_of_all += abs(current)
        count +=1
print("Maximum:", maximum)
print("Average:", sum_of_all/count)

/tmp/ipykernel_6276/1559176987.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  right_serie = vent_now[vent_now["local time"] == 0][vent_now["Id"] == i[1]["Id"]]


Maximum: 31.479015985564402
Average: 11.601944913893732


,error
count,402.000000
mean,11.601945
std,7.434132
min,0.000000
25%,5.697910
50%,11.232888
75%,19.032354
max,31.479016


In [17]:
vent_now

,Unnamed: 0,temperature,humidity,time,hum_out,temp_out,humidity_oben,temperature_oben,Id,local time
0,81,21.0,56,2023-03-09 06:40:42.552858112,88,10,50,19.1,2,0
1,82,21.0,56,2023-03-09 06:40:57.607806976,88,10,51,19.1,2,0
2,83,21.0,56,2023-03-09 06:43:37.784755968,88,10,51,19.1,2,2
3,84,21.0,56,2023-03-09 06:48:37.808173824,88,10,51,19.1,2,7
4,85,21.0,57,2023-03-09 06:51:42.940478976,88,10,51,19.1,2,11
5,86,21.2,57,2023-03-09 06:52:40.189434880,88,10,51,19.1,2,11
6,87,21.2,57,2023-03-09 06:53:37.823170048,88,10,51,19.1,2,12
7,662,21.2,55,2023-03-10 07:13:41.299113984,85,10,53,20.1,3,0
8,663,21.2,55,2023-03-10 07:18:41.333133824,85,11,53,20.1,3,5
9,1964,20.2,58,2023-03-12 12:14:20.774043136,83,11,44,19.6,4,0
